In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Test update
# hi it's me luke i'm also doing a test update, but like a second time though <3

/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv
/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv
/kaggle/input/nbme-score-clinical-patient-notes/features.csv
/kaggle/input/nbme-score-clinical-patient-notes/train.csv
/kaggle/input/nbme-score-clinical-patient-notes/test.csv


In [32]:
import pandas as pd
import pandas as np
import re
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

train_df = pd.read_csv("nbme-score-clinical-patient-notes/train.csv")
feature_df = pd.read_csv("nbme-score-clinical-patient-notes/features.csv")
notes_df = pd.read_csv("nbme-score-clinical-patient-notes/patient_notes.csv")

train_feature = pd.merge(train_df, feature_df, on=['case_num', 'feature_num'], how='left')
train_feature_notes = pd.merge(train_feature, notes_df, on=['pn_num', 'case_num'], how='left')

location = train_feature_notes.location
train_feature_notes.drop(columns=['location'], inplace=True)

print(train_feature_notes)

              id  case_num  pn_num  feature_num                                annotation                                       feature_text                                         pn_history
0      00016_000         0      16            0          ['dad with recent heart attcak']  Family-history-of-MI-OR-Family-history-of-myoc...  HPI: 17yo M presents with palpitations. Patien...
1      00016_001         0      16            1             ['mom with "thyroid disease']                 Family-history-of-thyroid-disorder  HPI: 17yo M presents with palpitations. Patien...
2      00016_002         0      16            2                        ['chest pressure']                                     Chest-pressure  HPI: 17yo M presents with palpitations. Patien...
3      00016_003         0      16            3      ['intermittent episodes', 'episode']                              Intermittent-symptoms  HPI: 17yo M presents with palpitations. Patien...
4      00016_004         0      16      

In [33]:
X_train, X_test, y_train, y_test = train_test_split(train_feature_notes, location, test_size=0.33, random_state=42)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                           num_labels=len(pd.unique(feature_df['feature_num'])))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=X_train,
                  eval_dataset=X_test)
ner_pipeline = pipeline("ner", grouped_entities=True, model=model)
features = ner_pipeline((["I ate a bottle of Adderall whole. Everyone clapped."]))

In [16]:
features

[[{'entity_group': 'MISC',
   'score': 0.7921942,
   'word': 'Adderall',
   'start': 18,
   'end': 26}]]